#  STILL I AM WORKING ON IT.
#not testesd yet, just started to translate
### Ingestion to COSMOSDB 


#### Set environment variables

In [ ]:
! pip install numpy
! pip install openai
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos
! pip install tenacity

In [21]:
import json
import datetime
import time

from azure.core.exceptions import AzureError
from azure.core.credentials import AzureKeyCredential
from azure.cosmos import exceptions, CosmosClient, PartitionKey
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.models import Vector
from azure.search.documents.indexes.models import (
    IndexingSchedule,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchField,
    SearchFieldDataType,
    SearchableField,
    SemanticConfiguration,
    SimpleField,
    PrioritizedFields,
    SemanticField,
    SemanticSettings,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    SearchIndexerDataSourceConnection
)

import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

import os
from dotenv import load_dotenv


In [ ]:

load_dotenv()

pg_host  = os.getenv("COSMOSDB_HOST")
if pg_host is None or pg_host == "":
    print("COSMOSDB_HOST environment variable not set.")
    exit()

pg_user  = os.getenv("COSMOSDB_USERNAME")
if pg_user is None or pg_user == "":
    print("COSMOSDB_USERNAME environment variable not set.")
    exit()

pg_password  = os.getenv("COSMOSDB_PASSWORD")
if pg_password is None or pg_password == "":
    print("COSMOSDB_PASSWORD environment variable not set.")
    exit()

db_name  = os.getenv("COSMOSDB_DATABASE")
if db_name is None or db_name == "":
    print("COSMOSDB_DATABASE environment variable not set.")
    exit()

text_table_name = 'text_sample'
doc_table_name = 'doc_sample'
image_table_name = 'image_sample'


In [ ]:
##secrets
##todo


### Cosmos containers


In [ ]:
from azure.cosmos import CosmosClient

# Your Cosmos DB connection parameters
cosmosdb_endpoint = "your_cosmosdb_endpoint"
cosmosdb_key = "your_cosmosdb_key"
cosmosdb_database_name = "your_database_name"
cosmosdb_container_name = "your_container_name"

####  Initialize the Cosmos DB client

In [ ]:
from azure.cosmos import CosmosClient

# Initialize the Cosmos DB client
client = CosmosClient(cosmos_db_api_endpoint, credential=cosmos_db_api_key)
database = client.get_database_client(cosmosdb_database_name)
container = database.get_container_client(cosmosdb_container_name)

# Function to create a container
def create_container(container_name, partition_key):
    container_definition = {
        "id": container_name,
        "partitionKey": {"paths": [f'/{partition_key}']},
    }
    container = database.create_container_if_not_exists(
        id=container_name, partition_key=partition_key
    )

    print(f"Container {container_name} created.")

# Create text_sample container
create_container(text_table_name, "id")

# Create doc_sample container
create_container(doc_table_name, "id")

# Create image_sample container
create_container(image_table_name, "id")


#### Ingest text sample with embeddings

In [ ]:
import pandas as pd
from azure.cosmos import CosmosClient


# Initialize the Cosmos DB client
client = CosmosClient(cosmos_db_api_endpoint, credential=cosmos_db_api_key)
database = client.get_database_client(cosmosdb_database_name)
container = database.get_container_client(cosmosdb_container_name)

# Read data from the JSON file
text_df = pd.read_json('../data/text/product_docs_embeddings.json')
records = text_df.to_dict(orient='records')

## this adaptation, the script uses the Cosmos DB Python SDK to create items in the Cosmos DB container. 
# The to_dict(orient='records') method is used to convert the Pandas DataFrame to a list of dictionaries, where each dictionary represents a record
# Function to insert data into Cosmos DB
def insert_data(container, records):
    for record in records:
        container.create_item(body=record)
    print("Text sample ingested.")

# Insert data into text_sample container
insert_data(container, records)


#### Ingest doc sample with embeddings

In [ ]:
import pandas as pd
from azure.cosmos import CosmosClient


# Initialize the Cosmos DB client
client = CosmosClient(cosmos_db_api_endpoint, credential=cosmos_db_api_key)
database = client.get_database_client(cosmosdb_database_name)
container = database.get_container_client(cosmosdb_container_name)

# Read data from the JSON file
doc_df = pd.read_json('../data/docs/employee_handbook_embeddings.json')
records = doc_df.to_dict(orient='records')

# Function to insert data into Cosmos DB
def insert_data(container, records):
    for record in records:
        container.create_item(body=record)
    print("Doc sample ingested.")

# Insert data into doc_sample container
insert_data(container, records)


#### Ingest image sample with embeddings

In [ ]:
import pandas as pd
from azure.cosmos import CosmosClient

# Initialize the Cosmos DB client
client = CosmosClient(cosmos_db_api_endpoint, credential=cosmos_db_api_key)
database = client.get_database_client(cosmosdb_database_name)
container = database.get_container_client(cosmosdb_container_name)

# Read data from the JSON file
image_df = pd.read_json('../data/images/images_embeddings.json')
records = image_df.to_dict(orient='records')

# Function to insert data into Cosmos DB
def insert_data(container, records):
    for record in records:
        container.create_item(body=record)
    print("Image sample ingested.")

# Insert data into image_sample container
insert_data(container, records)


#### Create IVFFlat Index

An IVFFlat index divides vectors into lists, and then searches a subset of those lists that are closest to the query vector. Details - https://github.com/pgvector/pgvector#ivfflat

In [ ]:
from azure.cosmos import CosmosClient

##Cosmos DB Python SDK to set the indexing policy for the specified fields. 
##Note that Cosmos DB is schema-agnostic, and you don't explicitly create indexes as you would in a relational database. 
##Instead, you define an indexing policy that specifies how Cosmos DB should index the data. 

# Initialize the Cosmos DB client
client = CosmosClient(cosmosdb_endpoint, cosmosdb_key)
database = client.get_database_client(cosmosdb_database_name)
container = database.get_container_client(cosmosdb_container_name)

def get_query_suffix(row_count):
    query_suffix = ''
    if row_count >= 1000:
        # Determine the number of lists based on the number of rows
        if row_count <= 1000000:
            lists = row_count / 1000
        else:
            lists = math.sqrt(row_count)
        query_suffix = f"WITH (lists = {int(lists)})"

    return query_suffix

# Function to create HNSW index for a container
def create_hnsw_index(container, field_name):
    indexing_policy = {
        "indexingMode": "consistent",
        "automatic": True,
        "includedPaths": [
            {
                "path": f'/{field_name}/*',
                "indexes": [
                    {
                        "kind": "Range",
                        "dataType": "Number",
                        "precision": -1
                    },
                    {
                        "kind": "Spatial",
                        "dataType": "Point"
                    }
                ]
            },
            {
                "path": f'/{field_name}',
                "indexes": [
                    {
                        "kind": "Range",
                        "dataType": "Number",
                        "precision": -1
                    },
                    {
                        "kind": "Spatial",
                        "dataType": "Point"
                    }
                ]
            }
        ],
        "excludedPaths": [
            {
                "path": "/*"
            }
        ],
        "spatialIndexes": [
            {
                "path": f'/{field_name}/*',
                "types": ["Geospatial"],
                "isAsync": False
            }
        ],
        "spatialDefault": {
            "types": ["Geospatial"],
            "isAsync": False
        },
        "indexingDirective": "Include"
    }

    container.replace_container(indexing_policy=indexing_policy)

# Create HNSW index for text_sample container
row_count = container.query_items("SELECT VALUE COUNT(1) FROM c", enable_cross_partition_query=True).next()
query_suffix = get_query_suffix(row_count)

create_hnsw_index(container, "title_vector")
create_hnsw_index(container, "content_vector")

print(f"HNSW index created for {cosmosdb_container_name}.")

##https://learn.microsoft.com/en-us/training/modules/search-azure-cosmos-db-sql-api-data-azure-cognitive-search/

##https://learn.microsoft.com/en-us/azure/search/search-howto-index-cosmosdb